# AI Medical Request Responder

This script utilizes the GPT API and the "langchain" module to respond to new medical requests within a medical center. It leverages embedding techniques to identify the most similar request in the database, providing contextually relevant responses.


In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts import PromptTemplate
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
os.environ["OPENAI_API_KEY"] = 'key'
model_id = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
hf_token = 'key'

### Data preprocessing



In [3]:
def clean_data_row(text): 
    split = text.split('[/INST]')
    patient = split[0].split('[INST]')[1].strip()
    response = split[1].strip()
    return patient, response

def clean_data(df):
    df[['Patient_Text', 'Doctor_Response']] = df['text'].apply(clean_data_row).apply(pd.Series)
    df['Doctor_Response'] = df['Doctor_Response'].str.replace('</s>', '').str.strip()
    df['Patient_Text'] = df.apply(lambda row: row['Patient_Text'].replace(row['Doctor_Response'], '').strip(), axis=1)
    df = df.drop(columns=['text'])
    return df

df = pd.read_csv('HealthCare.csv')
df = clean_data(df)

### Adding embedding col to the DB

In [5]:
def embed_sentence(text):
    return model.encode(text)

tqdm.pandas()
df['patients_embedding'] = df['Patient_Text'].progress_apply(embed_sentence)

100%|██████████| 12000/12000 [07:30<00:00, 26.65it/s]


In [6]:
df.to_csv("embeddeddata.csv")
embedded_df = pd.read_csv("embeddeddata.csv")

### Finding the most similar requests

In [7]:
def find_top3_similar_request(df, input_embedding):
    similarity_scores = []

    for index, row in df.iterrows():
        patient_embedding = row['patients_embedding']
        patient_embedding = np.array(np.matrix(patient_embedding)).ravel()
        similarity = cosine_similarity(patient_embedding.reshape(1, -1), input_embedding.reshape(1, -1))
        similarity_scores.append(similarity[0][0])

    embedded_df['similarity_score'] = similarity_scores

    top_indices = np.argsort(similarity_scores)[-3:][::-1] 
    top_similar_requests = df.iloc[top_indices]

    examples = []

    for index, row in top_similar_requests.iterrows():
        example = {
            'patient': row['Patient_Text'],
            'doctor': row['Doctor_Response']
        }
        examples.append(example)

    return top_similar_requests, examples



In [8]:
patient_text = "I hope you're well. I've been having persistent digestive issues for the past couple of months. These issues, while not debilitating, occur regularly and affect my daily routine. I've attempted some dietary adjustments with limited improvement. I'm uncertain if I've had a recent evaluation for this concern, so I'm reaching out to seek your guidance on possible causes and treatment options."

In [9]:
input_embedding = embed_sentence(patient_text)

top_requests, examples = find_top3_similar_request(df1, input_embedding)
print("Most Similar Request:")
print(top_requests.iloc[0]['Patient_Text'])

Most Similar Request:
I am now post gallbladder removal of about 4 months. The earliest they can get me in is in 2 weeks (go figure). I have been having an extreme issue with food passing through me way too quickly so much that I believe I am becoming malnourished - I am way too skinny and can t seem to gain weight. I only have pain immediately after I eat and once the food passes through me, I feel better. I found dumping syndrome and it fits me to a T. I can t find enough information on the internet to help me with diet - I am fit and eat quite healthy (mostly) and this is affecting my everyday life. I like to try natural remedies - what can I take to slow down digestion AND what can I eat? I found a site that listed fruits that aid digestion, but from reading, I need to stay away even from natural sugars (such as in fruit). I am in fear of landing a teaching job because I constantly need to be near the restroom within 5 minutes (or less) of eating. Thanks!


### Create a few shot prompt object to feed to the LLM

In [13]:
def create_a_prompt(examples):
    example_prompt = PromptTemplate(input_variables=["patient", "doctor"], template="Patient: {patient}\nDoctor: {doctor}")

    prompt = FewShotPromptTemplate(
        examples=examples,
        example_prompt=example_prompt,
        prefix='Act as a doctor on standby in healthcare app. you will recive an patient message and sent a response and recomendation for farter treatment based on previous respones. few examples:',
        suffix="Patient: {patient}",
        input_variables=["patient"]
    )
    return prompt

prompt = create_a_prompt(examples)
print(prompt.format(patient=patient_text))

Act as a doctor on standby in healthcare app. you will recive an patient message and sent a response and recomendation for farter treatment based on previous respones. few examples:

Patient: I am now post gallbladder removal of about 4 months. The earliest they can get me in is in 2 weeks (go figure). I have been having an extreme issue with food passing through me way too quickly so much that I believe I am becoming malnourished - I am way too skinny and can t seem to gain weight. I only have pain immediately after I eat and once the food passes through me, I feel better. I found dumping syndrome and it fits me to a T. I can t find enough information on the internet to help me with diet - I am fit and eat quite healthy (mostly) and this is affecting my everyday life. I like to try natural remedies - what can I take to slow down digestion AND what can I eat? I found a site that listed fruits that aid digestion, but from reading, I need to stay away even from natural sugars (such as in

In [14]:
def run_llm(prompt, input):
    llm = OpenAI(verbose=True, temperature=0.5)
    chain = LLMChain(llm=llm, prompt=prompt)
    
    output = chain.run(input)
    print(output)

### main function

In [16]:
def ask_the_doctor(request,df):
    input_embedding = embed_sentence(request)
    top_requests, examples = find_top3_similar_request(df, input_embedding)
    prompt = create_a_prompt(examples)
    run_llm(prompt, request)

request = patient_text
ask_the_doctor(request, embedded_df)



Doctor: Hi there, thank you for reaching out. It sounds like you have been having some persistent digestive issues that have been affecting your daily routine. It is important to identify the underlying cause of your symptoms in order to provide the most effective treatment. I recommend that you schedule an appointment with a gastroenterologist to evaluate your symptoms and determine the best course of treatment. The gastroenterologist can perform a variety of tests to help diagnose the cause of your symptoms, such as a blood test, an endoscopy, or a colonoscopy. In addition to seeking medical advice, I would recommend making some dietary adjustments, such as avoiding foods that are high in fat, and eating smaller, more frequent meals. If you have any further questions, please don't hesitate to reach out.
